# 95-865 Spring 2018 Mini-3 Final Exam [100 points]

In this final exam, you will compare how well two different methods do on a prediction task (solving word analogies):

* The first method (which we will call **Method 1**) is based on using pre-trained word embedding vectors.
* The second method (which we will call **Method 2**) is based on factoring a pointwise mutual information (PMI) matrix.

Both of these methods produce, for each word in a dictionary, a vector representation (so both methods produce so-called "word embeddings"). However, it is not clear which method should do better at solving word analogies.

As a remark: being able to solve word analogies is a basic task in natural language processing (and in fact, it has been used in GRE and SAT exams in the US to help gauge human English ability!).

**Note:** The completed notebook consumed ~3GB of RAM, and took ~4 minutes to execute. Keep this in mind when facing issues with running time or memory.

## Method 1

### (a) Load pre-trained word vectors [15 points]

The file `pretrained_word2vec_vectors.txt` contains 100-dimensional word vectors for $N$ words, trained using the gensim package and the word2vec model. The file is formatted as:

```
no_of_words vector_dimensionality
word vector-value vector-value ...
...
```

   1. Read the word vectors into a numpy array `w2v` with 100 columns and $N$ rows.
   2. Construct a list `w2v_words` of $N$ words ordered according to the row index of their vector in `w2v`.
   3. Normalize each vector by its L2 norm (hint: `numpy.linalg.norm`). After normalization, each row of `w2v` should have an L2 norm of 1.0.

In [1]:
import numpy as np

w2v_words = set([])
w2v_to_idx = {}
idx_to_w2v = {}
with open("pretrained_word2vec_vectors.txt", "r") as f:
    for idx, line in enumerate(f):
        if idx == 0:  # important: treat the header differently!
            nwords, ndim = map(int, line.split())
            w2v = np.zeros((nwords, ndim))  # the header tells us the shape of w2v
        else:
            fields = line.strip().split()
            
            word = fields[0]
            w2v_words.add(word)
            w2v_to_idx[word] = idx - 1  # subtract 1 to start counting words from 0 (ignoring the header)
            idx_to_w2v[idx - 1] = word
            
            vector = list(map(float, fields[1:]))
            w2v[idx-1, :] = vector

w2v = np.array([v/np.linalg.norm(v) for v in w2v])  # normalize each vector to have unit L2 norm

The code below can be used to test whether your normalization is correct:

In [2]:
assert np.allclose(np.array([np.linalg.norm(v) for v in w2v]),
                   np.ones(w2v.shape[0]))

### (b) Evaluate how well the this method does on predicting word analogies [20 points]

The `questions-words.txt` file contains word analogy tasks, one on each line. Consider the example:

```
Athens Greece Baghdad Iraq
```

This should be read as: "Athens is to Greece, as Baghdad is to ?". A good word embedding should predict the last word in the analogy well.

Let `a b c d` be the four words in an analogy task. Let `v` be the word embedding, so `v[w]` is the vector for word `w`. The accuracy of an embedding is computed as follows:

   1. Compute `pred = v[c] + (v[b] - v[a])`
   2. Find the word `y` in the corpus with vector `v[y]` second-closest to `pred` (in terms of the cosine distance). This is because the closes word to `pred` will most likely be `c`; hence, we use the second-closest instead.
   3. If `y` is equal to `d`, count the task as a success. If not, count it as a failure.

For the word embedding you loaded in part (a) and stored in the variable `w2v`, compute the number of successes using the three steps outlined above.

In [3]:
%%time
ntasks = 0
w2v_success = 0
with open("questions-words.txt", "r") as f:
    for line in f:
        a, b, c, d = line.strip().split()
        
        pred_w2v = w2v[w2v_to_idx[c]] + (w2v[w2v_to_idx[b]] - w2v[w2v_to_idx[a]])
        
        # interesting fact (that you didn't have to use): for vectors with unit L2 norm,
        # the cosine *similarity* between two vectors is actually just their dot product
        # so we just look for whichever word in `w2v` has the second largest dot product
        # with `pred_w2v` (you could have just used the cosine distance function
        # provided by scipy)
        predword_w2v_idx = np.argsort(np.dot(w2v, pred_w2v))[-2]
        predword_w2v = idx_to_w2v[predword_w2v_idx]
        if predword_w2v == d:
            w2v_success += 1

print('Number of successes:', w2v_success)

Number of successes: 38
CPU times: user 16.6 s, sys: 286 ms, total: 16.8 s
Wall time: 13 s


## Method 2

For this second method, we will learn a word embedding from scratch using an existing text corpus, computing PMI values (and storing it into a 2D table, i.e., a matrix), and factoring the PMI matrix to obtain word embeddings. We can then use the word embeddings to, just as in part (b) for Method 1 above, predict word analogies.

### (a) Load text corpus [10 points]

The file `brown_corp.txt` contains the Brown University Standard Corpus of Present-Day American English: the first modern, computer-readable text corpus.

The file has a single line with a list of words separated by spaces. Sentences are separated by a period with spaces on either side, like " . ".

   1. Construct a list `brown_corpus` that is a list of lists. Each element of `brown_corpus` is a sentence (as a list of words).
   2. Discard any word not in `w2v_words`.
   3. Print the final size of the Brown corpus vocabulary.
   4. Print the set of words in the Brown corpus vocabulary that are not in `w2v_words`.

In [4]:
line = open("brown_corp.txt", "r").readlines()[0].strip()
sentences = line.split(" . ")
brown_corpus = []
brown_words = set([])
for sentence in sentences:
    words = sentence.split()
    words = [w for w in words if w in w2v_words]  # only keep words that are also in `w2v_words`
    brown_corpus.append(words)
    for word in words:
        brown_words.add(word)

### (b) Construct word embeddings via PMI matrix factorization [30 points]

Refer to the recitation of January 26, 2018 for hints (in case you do not have your notes handy, at the end of this problem's instructions, we have links to the recitation notes).

   1. Construct a matrix `PMI` where each cell contains the pointwise mutual information between a pair of words in the preprocessed Brown corpus. Bigrams are counted for any two words occurring in the same sentence. (20 points)
   2. Factorize `PMI` using SVD (`scipy.sparse.linalg.svds`) to obtain 100-dimensional vectors for each word, stored in a matrix `U`. Normalize each vector by its L2 norm. (10 points)
   
The PMI value for a pair of words (x, y) is (note that the recitation has a typo in this formula): $$ \textrm{PMI}(x,y) = \textrm{log}(\frac{p(x,y)}{p(x)p(y)}) $$ where $p(x,y)$ is the probability of the bigram $x y$ occurring in the corpus, and $p(x)$ is the probability of the unigram $x$ occurring in the corpus.

Be careful during division: dividing two integers may result in a 0, if none of them are first converted to `float`s.

**Notes from the recitation on January 26, 2018:**

* PDF: http://www.eyeshalfclosed.com/teaching/95865-recitation-word2vec_as_PMI.pdf
* Jupyter notebook: https://gist.github.com/emaadmanzoor/1d06e0751a3f7d39bc6814941b37531d
* Dataset: https://www.kaggle.com/hacker-news/hacker-news-posts/downloads/HN_posts_year_to_Sep_26_2016.csv


In [5]:
%%time
from scipy.sparse.linalg import svds, norm
from scipy.sparse import csc_matrix
from collections import Counter
from itertools import combinations

# unigram and bigram counts
cx = Counter()
cxy = Counter()
for text in brown_corpus:
    for x in text:
        cx[x] += 1.
    for x, y in map(sorted, combinations(text, 2)):
        cxy[(x, y)] += 1.

CPU times: user 22.8 s, sys: 572 ms, total: 23.4 s
Wall time: 24.2 s


In [6]:
# sums
sx = float(sum(cx.values()))
sxy = float(sum(cxy.values()))

In [7]:
%%time
# unigram - vector lookup
x2i, i2x = {}, {}
for i, x in enumerate(cx.keys()):
    x2i[x] = i
    i2x[i] = x

CPU times: user 8.86 ms, sys: 2.31 ms, total: 11.2 ms
Wall time: 13.5 ms


In [8]:
%%time
# PMI
from math import log
pmi_samples = Counter()
data, rows, cols = [], [], []
for (x, y), n in cxy.items():
    rows.append(x2i[x])
    cols.append(x2i[y])
    data.append(log((n / sxy) / (cx[x] / sx) / (cx[y] / sx)))
    pmi_samples[(x, y)] = data[-1]
PMI = csc_matrix((data, (rows, cols)))

CPU times: user 8.89 s, sys: 598 ms, total: 9.49 s
Wall time: 9.99 s


In [9]:
%%time 
# SVD
from scipy.sparse.linalg import svds
U, _, _ = svds(PMI, k=100)

CPU times: user 14 s, sys: 290 ms, total: 14.3 s
Wall time: 7.84 s


In [10]:
U = np.array([v/np.linalg.norm(v) for v in U])  # normalize each row to have unit L2 norm

The code below can be used to test whether your normalization is correct:

In [11]:
assert np.allclose(np.array([np.linalg.norm(v) for v in U]),
                   np.ones(U.shape[0]))

### (c) Evaluate how well the this method does on predicting word analogies [20 points]

Please repeat part (b) from Method 1 except now using the word embedding stored in variable `U` computed for this second PMI-based method. Once again, print the total number of successes. **Then clearly state which method has a higher number of successes in predicting word analogies.**

In [12]:
%%time
ntasks = 0
pmi_success = 0
w2v_success = 0
with open("questions-words.txt", "r") as f:
    for line in f:
        a, b, c, d = line.strip().split()
        
        pred_pmi = U[x2i[c]] + (U[x2i[b]] - U[x2i[a]])
 
        predword_pmi_idx = np.argsort(np.dot(U, pred_pmi))[-2]
        predword_pmi = i2x[predword_pmi_idx]
        if predword_pmi == d:
            pmi_success += 1
        
print('Number of successes:', pmi_success)

Number of successes: 165
CPU times: user 14.8 s, sys: 208 ms, total: 15 s
Wall time: 11.4 s


**Your answer for which method does better** (just say either Method 1 or Method 2): Method 2

### (d) Briefly explain why we cannot easily do k-fold cross validation with either of the methods here. [5 points]

**Your answer:** In the first method, we do not know what the training data are that was used for training the model, and as a result have no way of repeating this training procedure for different folds of the data. In the second method, we do not have any sort of ground truth word analogies associated with different parts of the Brown University text corpus. In order to do k-fold cross validation, we would need to be able to split up the training data into folds, where each fold has both training and validation sets. It's not clear how one would set up the validation sets in this case!